# 如何分析算法

设计数据结构与算法根本上说是为了在有限的资源限制下达到算法的预期目的.通常制约算法的条件是`时间`和`内存空间`,但最主要的制约是时间.

先来一个helloworld,我们以此为例看看究竟如何分析算法


> 例1: 设有N个整数要确定其中的第k个最大数.

这个例子是一个选择问题(selection problem).很自然的我们会有一个最直观的方案--把这些数排个序(快排/冒泡),然后取出第`k`个最大的就行了.

In [1]:
def findK(k,target):
    new = sorted(target)
    return new[k-1]


In [2]:
findK(5,[4,3,2,1,5,9,8,7,6])

5

再进一步思考,我们如果处理一组相当大量的数,那全部排序会花费大量时间(最好的也要时间复杂度$O(N*log_2N)$),有没有更好的方法可以减少这个时间消耗呢?毕竟我们只要第n个数而已,这么思考的话,我们可以先给出一块排名长度内存空间`cache`,直接遍历一次数组,每个数在`cache`中进行比较插入.这样一遍下来就可以获得前`k`个排名了.这种方式要运算的次数就降到了$ N*k $(时间复杂度$O(N*k)$)代价就是多使用了一个`cache`的空间.

In [3]:
import collections


def findKcache(k,target):
    cache = collections.deque(maxlen=k)
    for i in target:
        if len(cache) <k:
            cache.appendleft(i)
            cache = collections.deque(sorted(cache),maxlen=k)
        else:
            if cache[-1]>i> cache[0]:
                cache.appendleft(i)
                cache = collections.deque(sorted(cache),maxlen=k)
    return cache[-1]

In [4]:
findKcache(5,[4,3,2,1,5,9,8,7,6])

5

这种方法因为缓存已经用于存储结果了.这时我们很自然的会问,这两种那个更好?似乎直觉上感觉第二个更好些,但其实也不尽然,第二种的效率取决于K的大小,当K的值为N时时间复杂度就是$O(N*N)$了自然比第一种都遭.我们来实际试一下

In [5]:
import random
target1 = [random.random() for i in range(10)]
target2 = [random.random() for i in range(100)]
target3 = [random.random() for i in range(1000)]
target4 = [random.random() for i in range(10000)]
target5 = [random.random() for i in range(100000)]

In [6]:
%timeit findK(5,target1)

668 ns ± 29.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [7]:
%timeit findKcache(5,target1)

11.2 µs ± 536 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [8]:
%timeit findK(5,target2)

5.83 µs ± 433 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
%timeit findKcache(5,target2)

33.8 µs ± 1.57 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
%timeit findK(5,target3)

103 µs ± 3.96 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [11]:
%timeit findKcache(5,target3)

195 µs ± 5.26 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
%timeit findK(5,target4)

1.53 ms ± 72.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%timeit findKcache(5,target4)

1.67 ms ± 53.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
%timeit findK(5,target5)

25.2 ms ± 2.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%timeit findKcache(5,target5)

18.2 ms ± 119 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


在数据规模到达一定程度,且固定查找第5个时,似乎第二种方法性能确实会超越第一种.

事实上第一种方法在业务上更加灵活,因为它可以得到K为任意值时的结果,而第2种在比如万个数据取前10,前5这样的场景中就比较靠谱.而如果是大规模数据的求第K大值问题,一种比较稳定的算法是使用堆,这个会在后面讲到堆的时候详细说明.

上面的例子是为了说名分析算法的一个一般流程

1. 估算消耗
2. 结合场景评估效果

事实上通常写一个程序本身不难,难在当规模扩大时如何结合实际场景选择实现方式.

## 数学知识复习

这部分列出需要记住的或者可以推导出的基本公式.

### 指数

$$ X^AX^B = X^{A+B} $$
$$ \frac {X^A}{X^B} = X^{A-B} $$

$$(X^A)^B = X^{AB}$$

$$ X^N+X^N = 2X^N \neq X^{2N} $$

$$ 2^N+2^N = 2^{N+1} $$

### 对数

在计算科学中,除非有特殊申明,否则所有的对数都是以2位底.

定义: 在$X>0$的情况下$X^A = B$当且仅当$log_XB=A$

由此可以得到几个方便的等式.

> **定理** $$ log_AB = \frac {log_CB}{log_CA} ; C>0 $$
> **证明:**
> 令:$$X = log_CB,Y = log_CA,Z = log_AB$$
> 由定义可得:$$C^X=B,C^Y = A,A^Z=B$$
> $$C^X=A^Z=B$$
> $$C^X=(C^Y)^Z=B$$
> $$C^X=(C)^{YZ}=B$$
> 由此可得: $$X = YZ$$
> $$Z = \frac {X}{Y}$$
> 即: $$ log_AB = \frac {log_CB}{log_CA}$$

> **定理** $$logAB=logA+logB$$
> **证明:**
> 令: $$X = logA,Y = logB,Z = logAB$$
> 由定义可得: $$2^X=A,2^Y = B,2^Z=AB$$
> $$2^X*2^Y = 2^Z=AB$$
> $$2^{X+Y} = 2^Z$$
> 由此可得: $${X+Y} = Z$$
> 即: $$logAB=logA+logB$$

其他几个也一样可以这么推导得到

> **定理** $$logA/B=logA-logB$$
> **证明:**
> 令: $$X = logA,Y = logB,Z = log\frac{A}{B}$$
> 由定义可得: $$2^X=A,2^Y = B,2^Z=\frac{A}{B}$$
> $$\frac{2^X}{2^Y} = 2^Z=\frac{A}{B}$$
> $$2^{X-Y} = 2^Z$$
> 由此可得: $${X-Y} = Z$$
> 即: $$log\frac{A}{B}=logA-logB$$

> **定理** $$log(A^B)=BlogA$$
> **证明:**
> 令: $$X = logA,Y = log(A^B)$$
> 由定义可得: $$ 2^X=A,2^Y=A^B $$
> $$2^Y=(2^X)^B$$
> $$2^Y=(2)^{X*B}$$
> 由此可得: $$Y = BX$$
> 即: $$log(A^B)=BlogA$$

> **定理** $$logX <X ; X>0 $$
> **证明:**
> 我们可以简单的求两者的差:
> $$Y = X-logX$$假设他们会相交即令Y=0
> 求其导数$$Y' = 1 -\frac {1}{xln2}$$
> 这个函数中$ln2$是一个常数,求$$ 1- \frac{1}{xln2} =0 ; x = \frac 1{ln2}$$ 
> 也就是说$$x = \frac 1{ln2}是 Y = X-logX 的极值点;$$
> 求得$$Y'' = \frac 1 {(xln2)^2}* ln2 = \frac 1 {(x^2ln2)}$$ 
> 所以$Y''$恒大于0.由此可得在$Y' = 1 -\frac {1}{xln2}$点是最小值
>
> Y中带入$$x = \frac 1{ln2}$$
> 有: $$Y = \frac 1{ln2}-log(\frac 1{ln2})=1.9714..>0$$
> 因此有$$logX <X ; X>0$$


### 级数

最常见的有两个公式:

> 公式: $$\sum_\mit{i=0}^N A^i = \frac {A^{N+1}-1}{A-1}$$,
> 常用的当$0<A<1$时有:$$\sum_\mit{i=0}^N A^i \leq \frac 1{1-A}$$
> 常用的当$A=2$时有:$$\sum_\mit{i=0}^N 2^i = 2^{N+1}-1$$

> 公式: $$\sum_\mit{i=1}^\infty \frac i{2^i} = 2$$

> 公式: $$\sum_\mit{i=1}^N i^k \approx \frac{N^{k+1}}{|k+1|} , k \neq -1$$
> 常用的当k=1时有: $$\sum_\mit{i=1}^N i =  \frac {N(N+1)}{2} \approx \frac{N^2}{2}$$
> 常用的当k=2时有: $$\sum_\mit{i=1}^N i^2 =  \frac {N(N+1)(2N+1)}{6} \approx \frac{N^3}{3}$$

> 公式: $$H_N = \sum_\mit{i=1}^N \frac {1}{i} \approx lnN$$,$H_N$被称为调和数,其和叫调和和,这个式子中误差趋向于$\gamma \approx 0.57721566$,这个值被称为欧拉常数.

> 公式:$$\sum_\mit{i=1}^N f(N) = Nf(N)$$

> 公式:$$\sum_\mit{i=n_0}^N f(i) = \sum_\mit{i=1}^N f(i) - \sum_\mit{i=1}^{n_0-1} f(i)$$

## 如何估计一个程序所需要的时间


一般来说我们评估一个算法必然会以其规模增加的倍数N作为参考.最关注的是其相对增长率,比如当N较小时$1000N$要比$N^2$大,但$N^2$增长更快,而1000只是一个常数,也就是高阶与低阶的区别.我们在估计一个程序所需时间时一般都会忽略常数项和低阶项.通常我们使用`大O记法`记录时间复杂度,如:$O(N^2)$,通常来讲我们希望算法的时间复杂度是$O(N)$,次一点的是$O(NlogN)$,在一些极端情况下,我们可以有限的使用时间复杂度为$O(N^2)$的算法.高于这些的话时间过长基本是不能接受的.

### 估算的方法
1. 数for循环,一次for循环至多是$O(N)$,k个嵌套的for循环则至多是$O(N^k)$,而外层循环个个数不重要,只要知道嵌套最深的for循环就可以知道时间复杂度至多是多少了.

2. 当数据增大N倍时,时间损耗扩大logN倍这种情况通常出现在二分法,即每次迭代会将结果范围剔除一半.